# TensorBoard
In this tutorial, we are going to implement a neural network with fully-connected layers to perform classification, visualize the model and plot the loss and gradients by using a tensorboard.

![alt text](jpg/tensorboard.jpg "model image")

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data


mnist = input_data.read_data_sets("./mnist", one_hot=True)
x_train = mnist.train.images
y_train = mnist.train.labels
x_test = mnist.test.images
y_test = mnist.test.labels

print "x_train: ", x_train.shape
print "y_train: ", y_train.shape
print "x_test: ", x_test.shape
print "y_test: ", y_test.shape

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
x_train:  (55000, 784)
y_train:  (55000, 10)
x_test:  (10000, 784)
y_test:  (10000, 10)


## Define Network

In [2]:
def fully_connected(x, dim_in, dim_out, name):
    with tf.variable_scope(name) as scope:
        # create variables
        w = tf.get_variable('w', shape=[dim_in, dim_out], 
                            initializer=tf.random_uniform_initializer(minval=-0.1, maxval=0.1))
        b = tf.get_variable('b', shape=[dim_out])
        
        # create operations
        out = tf.matmul(x, w) + b
        
        return out    

In [3]:
# Create model
def neural_network(x, dim_in=784, dim_h=500, dim_out=10):
    # 1st hidden layer with ReLU
    h1 = fully_connected(x, dim_in, dim_h, name='hidden_layer_1')
    h1 = tf.nn.relu(h1)
    
    # 2nd hidden layer with ReLU
    h2 = fully_connected(h1, dim_h, dim_h, name='hidden_layer_2')
    h2 = tf.nn.relu(h2)
    
    # output layer with linear
    out = fully_connected(h2, dim_h, dim_out, name='output_layer')
    
    return out

## Construct graph

In [5]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

# Construct model with default value
out = neural_network(x)

## Loss, Optimizer and Summary

In [6]:
# loss 
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(out, y))

# accuracy
with tf.name_scope('accuracy'):
    pred = tf.argmax(out, 1)
    target = tf.argmax(y, 1)
    correct_pred = tf.equal(pred, target)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    
# train op
with tf.name_scope('optimizer'):
    optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)
    grads = tf.gradients(loss, tf.trainable_variables())
    grads_and_vars = list(zip(grads, tf.trainable_variables()))
    train_op = optimizer.apply_gradients(grads_and_vars=grads_and_vars)

# add summary op   
tf.scalar_summary('batch_loss', loss)
for var in tf.trainable_variables():
    tf.histogram_summary(var.op.name, var)
for grad, var in grads_and_vars:
    tf.histogram_summary(var.op.name+'/gradient', grad)

summary_op = tf.merge_all_summaries() 

## Session: train and test model

In [7]:
batch_size = 100
log_path = 'log/'

# launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    # initialize tensor variables
    tf.initialize_all_variables().run()
    summary_writer = tf.train.SummaryWriter(log_path, graph=tf.get_default_graph())
    # training cycle
    for epoch in range(15):
        avg_loss = 0.
        n_iters_per_epoch = int(mnist.train.num_examples / batch_size)
        # loop over all batches
        for i in range(n_iters_per_epoch):
            x_batch, y_batch = mnist.train.next_batch(batch_size)
            # run optimization op (backprop) and loss op (to get loss value)
            feed_dict={x: x_batch, y: y_batch}
            _, c = sess.run([train_op, loss], feed_dict=feed_dict)
            # compute average loss
            avg_loss += c / n_iters_per_epoch
            
            if i % 10 == 0:
                summary = sess.run(summary_op, feed_dict)
                summary_writer.add_summary(summary, epoch*n_iters_per_epoch + i)
        print "Epoch %d, Loss: %.3f"% (epoch+1, avg_loss)
    print "Finished training!"
    
    print ""
    print "Test accuracy:", sess.run(accuracy, {x: mnist.test.images, y: mnist.test.labels})

Epoch 1, Loss: 0.573
Epoch 2, Loss: 0.108
Epoch 3, Loss: 0.065
Epoch 4, Loss: 0.044
Epoch 5, Loss: 0.032
Epoch 6, Loss: 0.023
Epoch 7, Loss: 0.019
Epoch 8, Loss: 0.015
Epoch 9, Loss: 0.012
Epoch 10, Loss: 0.010
Epoch 11, Loss: 0.008
Epoch 12, Loss: 0.008
Epoch 13, Loss: 0.007
Epoch 14, Loss: 0.006
Epoch 15, Loss: 0.005
Finished training!

Test accuracy: 0.981


<br>
## Execute the TensorBoard
To execute the tensorboard, open the new terminal, run command below and open http://localhost:6005/ into your web browser.
```bash
$ tensorboard --logdir='./log' --port=6005 
```